# **Exploring Static Data**
In this notebook, we will explore the static data provided by ATAC. Below is the entity relationship diagram for the static data. Here is the [link](https://drawsql.app/teams/atacontitan/diagrams/static-data) to the drawsql diagram.

![Static Data ERD](figures/er_diagram.png)

In [7]:
# Libraries
import pandas as pd
from google.transit import gtfs_realtime_pb2
feed = gtfs_realtime_pb2.FeedMessage()

# Static data source
src = 'data/rome_static_gtfs_test'

In [15]:
agency = pd.read_csv(f'{src}/agency.txt')
calendar = pd.read_csv(f'{src}/calendar.txt')
calendar_dates = pd.read_csv(f'{src}/calendar_dates.txt')
routes = pd.read_csv(f'{src}/routes.txt')
shapes = pd.read_csv(f'{src}/shapes.txt')
stop_times = pd.read_csv(f'{src}/stop_times.txt', nrows=1000)
stops = pd.read_csv(f'{src}/stops.txt')
trips = pd.read_csv(f'{src}/trips.txt')

In [55]:
print(f'Unique ID for agency is "agency_id": {agency["agency_id"].duplicated().sum()}')
print(f'Unique ID for calendar is "service_id": {calendar["service_id"].duplicated().sum()}')
print(f'Unique ID for routes is "route_id": {routes["route_id"].duplicated().sum()}')
print(f'Unique ID for shapes is "shape_id" and "shape_pt_sequence": {shapes[["shape_id", "shape_pt_sequence"]].duplicated().sum()}')
print(f'Unique ID for stops is "stop_id": {stops["stop_id"].duplicated().sum()}')
print(f'Unique ID for trips is "trip_id": {trips["trip_id"].duplicated().sum()}')

Unique ID for agency is "agency_id": 0
Unique ID for calendar is "service_id": 0
Unique ID for routes is "route_id": 0
Unique ID for shapes is "shape_id" and "shape_pt_sequence": 0
Unique ID for stops is "stop_id": 0
Unique ID for trips is "trip_id": 0
